In [ ]:
# cuny-mri.flywheel.io:djEhbcLgXzMrzELz7XHKFCXfZVSsl9cYDDRFs9Hhue6mjBowm8o_0J06Q
# fw login cuny-mri.flywheel.io:djEhbcLgXzMrzELz7XHKFCXfZVSsl9cYDDRFs9Hhue6mjBowm8o_0J06Q

In [11]:
import os
import glob
import flywheel
import logging
import re
import subprocess
import zipfile
import datetime
import gc

In [12]:
#Initialize the logger
logging.basicConfig(filename='download_mrs.log', level=logging.INFO)
logger = logging.getLogger()

In [13]:
#fw = flywheel.Client('cuny-mri.flywheel.io:djEhbcLgXzMrzELz7XHKFCXfZVSsl9cYDDRFs9Hhue6mjBowm8o_0J06Q')

In [14]:
# Get API Key from local environment variables and initialize the Flywheel client
#api_key = os.environ.get('API_KEY')
fw = flywheel.Client("cuny-mri.flywheel.io:djEhbcLgXzMrzELz7XHKFCXfZVSsl9cYDDRFs9Hhue6mjBowm8o_0J06Q")

In [15]:
# Path where the MRS data will be downloaded
root_path = '/Users/jacekdmochowski/PROJECTS/fus/data/'

# Get the Flywheel Project
project = fw.lookup('jacek/FusRelabeledCopy')

In [16]:
def download_all_mrs(root_data):
    #Get the group label
    group_label = project.group
    
    #Iterate through all the files in the 'ACTIVE' and 'SHAM' sessions in the project
    for subject in project.subjects():
        
        for session in subject.sessions():
            
            # Input the session labels with MRS data.
            
            if session.label == 'ACTIVE' or session.label == 'SHAM': 
                
                for acquisition in session.acquisitions():
                    
                    acq_label = acquisition.label
                    
                    #Check if the acquisition label starts with 'mrs'
                    
                    if acq_label.startswith('mrs'):
                        
                        #Create the directory using the Flywheel hierarchy
                        path = f'{root_path}\\{group_label}\\{project.label}\\{subject.label}\\{session.label}\\{acq_label}'
                        os.makedirs(path, exist_ok=True)
                        
                        for file in acquisition.files:
                            
                            #Find the dicom file in the mrs acquisition
                            if file.type == 'dicom':
                                
                                if file not in os.walk(path):
                                    
                                    try:
                                        #Download the file from the acquisition.
                                        
                                        file_path = f'{path}\\{file.name}'
                                        
                                        file_name = f'sub-{subject.label}_ses-{session.label}_acq-{acq_label}'
                                        
                                        acquisition.download_file(file.name, file_path)
                                        
                                        # Open the zip file
                                        with zipfile.ZipFile(file_path, 'r') as zip_ref:
                                            #Extract the contents of the zip file to the path
                                            zip_ref.extractall(path)
                                            
                                        #Delete compressed zip file
                                        os.remove(file_path)
                                        
                                        #Rename the unzipped folder to the new file name
                                        unzipped_folder = glob.glob(f'{path}\\*')[0]
                                        os.rename(unzipped_folder, f'{path}\\{file_name}')
                                        
                                        logger.info(f'Downloaded {file.name} from {acq_label} in {session.label} for {subject.label}')
                                        
                                        print(f'Downloaded {file.name} from {acq_label} in {session.label} for {subject.label}')
                                        
                                    except Exception as e:
                                        logger.error(f'Error downloading {file.name} from {acq_label} in {session.label} for {subject.label}. {e}')
                                break


In [17]:
download_all_mrs(root_path)